## Results
Fit models to real-world data, as shown in @fig-fit.

In [ ]:
import warnings
from typing import List
import numpy as np
import pandas as pd
from datetime import datetime
import nevergrad as ng
from IPython.display import Markdown
from collections import namedtuple

from emu_renewal.renew import RenewalModel, TruncRenewalModel
from emu_renewal.outputs import plot_output_fit

In [ ]:
# Data and fixed parameters
run_in = 30
n_process_periods = 12
raw_data = pd.read_csv('https://github.com/monash-emu/wpro_working/raw/main/data/new_cases.csv', index_col=0)['MYS']
raw_data.index = pd.to_datetime(raw_data.index)
mys_data = raw_data.loc[datetime(2021, 3, 1): datetime(2021, 11, 1)].reset_index()['MYS']
mys_data.index += run_in
n_times = len(mys_data) + run_in
calib_kwargs = {'pop': 33e6, 'n_times': n_times, 'run_in': run_in, 'targets': mys_data}
fixed_param_desc = '### Fixed parameter values\n ' \
    f'The target population is initialised as {str(int(calib_kwargs["pop"]))} susceptible persons. ' \
    f'The simulation runs for a run-in period of {run_in} days before comparison against the calibration data commences.\n'

In [ ]:
# Define parameter ranges
scalar_req = {
    'Generation time mean (days)' : {'init': 5.0, 'lower': 0.1, 'upper': 14.0},
    'Generation time standard deviation (days)': {'init': 5.0, 'lower': 2.5, 'upper': 8.0},
    'Case detection proportion': {'init': 0.06, 'lower': 0.04, 'upper': 0.2},
    'Log starting seed rate': {'init': np.log(1e4), 'lower': np.log(5e3), 'upper': np.log(2e4)},
}
params_df = pd.DataFrame(scalar_req).transpose()
proc_req = {'init': 0.0, 'lower': -2.0, 'upper': 2.0}

In [ ]:
# Get the renewal model object - choosing between standard and with truncated generation times
model = RenewalModel(calib_kwargs['pop'], n_times, run_in, n_process_periods)

# long_gen_densities = model.dens_obj.get_densities(
#     100, 
#     scalar_req['Generation time mean (days)']['upper'], 
#     scalar_req['Generation time standard deviation (days)']['upper'],
# )
# gen_times_end = np.argmax(long_gen_densities.cumsum() > 0.999)
# model = TruncRenewalModel(calib_kwargs['pop'], n_times, run_in, n_process_periods, gen_times_end)

In [ ]:
#| warning: false
calib_desc = '\n\n### Calibration targets\nThe model described above was fit to the target data ' \
    'to minimise the square of the difference between the modelled notification rate ' \
    'and the reported case numbers at each date considered. ' \
    'Modelled notifications are calculated as the product of modelled incidence and the ' \
    '(constant through time) case detection proportion. '


def calib_func(parameters: List[float], pop: int, n_times: int, run_in: int, targets: dict) -> float:
    gen_time_mean, gen_time_sd, cdr, seed, *process = parameters
    incidence = model.func(gen_time_mean, gen_time_sd, process, seed).incidence
    return sum([(incidence[t] * cdr - d) ** 2 for t, d in targets.items()])

def obj_func(gen_time_mean, gen_time_sd, cdr_param, seed, proc_params):
    return calib_func([gen_time_mean, gen_time_sd, cdr_param, seed] + list(proc_params), **calib_kwargs)

scalar_params = [ng.p.Scalar(**row).set_name(idx) for idx, row in params_df.iterrows()]
process_params = ng.p.Array(init=[proc_req['init']] * n_process_periods, lower=proc_req['lower'], upper=proc_req['upper']).set_name('Log non-mechanistic process values')
instrum = ng.p.Instrumentation(*scalar_params, process_params)
optimizer = ng.optimizers.NGOpt(parametrization=instrum, budget=100)
ngopt_result = optimizer.minimize(obj_func).value[0]

In [ ]:
# Pseudo loglikelihood functions

def calib_func_ll(parameters: List[float], pop: int, n_times: int, run_in: int, targets: dict) -> float:
    gen_time_mean, gen_time_sd, cdr, seed, *process = parameters
    incidence = model.func(gen_time_mean, gen_time_sd, process, seed).incidence
    return 0.0 - sum([(incidence[t] * cdr - d) ** 2 for t, d in targets.items()])

# This is what we'll actually call from pymc
def obj_func_ll(gen_time_mean, gen_time_sd, cdr_param, seed, proc_params):
    return calib_func_ll([gen_time_mean, gen_time_sd, cdr_param, seed] + list(proc_params), **calib_kwargs)


In [ ]:
from estival import priors as esp

In [ ]:
priors = [esp.UniformPrior(k, (v["lower"], v["upper"])) for k,v in params_df.iterrows()]
priors.append(esp.UniformPrior('Log non-mechanistic process values', (proc_req["lower"],proc_req["upper"]), size=n_process_periods))

In [ ]:
import pymc as pm
import pytensor.tensor as pt

def get_wrapped_ll(priors, llfunc):
    """_summary_

    Args:
        bcm: The model to wrap

    Returns:
        A wrapped pytensor op for use in pymc
    """

    prior_types = []
    for prior in priors:
        if prior.size > 1:
            prior_types.append(pt.dvector)
        else:
            prior_types.append(pt.dscalar)

    # define a pytensor Op for our likelihood function
    class CustomLogLike(pt.Op):
        """
        Specify what type of object will be passed and returned to the Op when it is
        called. In our case we will be passing it a vector of values (the parameters
        that define our model) and returning a single "scalar" value (the
        log-likelihood)
        """

        itypes = prior_types

        # [pt.dscalar] * len(
        #    bcm.priors
        # )  # [pt.dvector]  # expects a vector of parameter values when called
        otypes = [pt.dscalar]  # outputs a single scalar value (the log likelihood)

        def __init__(self):
            """
            Initialise the Op with various things that our log-likelihood function
            requires. Below are the things that are needed in this particular
            example.

            Parameters
            ----------
            bcm: BayesianCompartmentalModel
            """

            # Capture the BayesianCompartmentalModel
            self.llfunc = llfunc

        def perform(self, node, inputs, outputs):
            params = inputs
            #kwargs = {k: params[i] for i, k in enumerate(self.bcm.priors)}

            # call the log-likelihood function
            logl = self.llfunc(*params)

            outputs[0][0] = np.array(logl)  # output the log-likelihood

    return CustomLogLike


def use_model(priors, llfunc) -> list:
    """Use a given BayesianCompartmentalModel for pymc sampling
    This should be called inside a model context like so

    with pm.Model():
        variables = use_model(bcm)
        pm.sample(step=[pm.DEMetropolis(variables)])

    Args:
        bcm: The BCM to use for sampling
        include_ll: Include loglikelihood in the sample outputs

    Returns:
        The list of variables to be passed to a sampler step
    """
    logl = get_wrapped_ll(priors, llfunc)()

    pymc_priors = [p.to_pymc() for p in priors]
    invars = [pt.as_tensor_variable(v) for v in pymc_priors]

    # use a Potential to "call" the Op and include it in the logp computation
    ll = logl(*invars)
    pot = pm.Potential("loglikelihood", ll)

    return pymc_priors



In [ ]:
with pm.Model() as pmm:
    variables = use_model(priors, obj_func_ll)
    idata = pm.sample(step=[pm.DEMetropolisZ(variables)])


In [ ]:
import arviz as az

In [ ]:
az.summary(idata)

In [ ]:
mean_posterior_params = mpp = np.array(az.summary(idata)["mean"])

In [ ]:
incidence = model.func(mpp[0], mpp[1], mpp[4:], mpp[3]).incidence * mpp[2]

In [ ]:
pd.Series(incidence).plot()
calib_kwargs["targets"].plot()

In [ ]:
#| label: fig-fit
#| fig-cap: "Optimisation to sample data from Malaysia"
model_result = model.func(ngopt_result[0], ngopt_result[1], ngopt_result[4], ngopt_result[3])
fig = plot_output_fit(mys_data, model_result, n_times, cdr=ngopt_result[2])
fig.write_image('results_fig.svg')

![Results of fitting to data from Malaysia.](results_fig.svg){#fig-fit}

## Methods

In [ ]:
Markdown(fixed_param_desc) 

In [ ]:
Markdown(model.get_description())

In [ ]:
Markdown(calib_desc)

### Calibrated parameters
The parameters used in the optimisation presented here are presented in @tbl-params.

In [ ]:
params_df.loc['Log non-mechanistic process values'] = proc_req
params_df.columns = ['Starting point', 'Lower limit', 'Upper limit']
Markdown(params_df.to_markdown() + '\n : Calibration parameters table {#tbl-params}')

In [ ]:
evidence_table = pd.DataFrame(index=params_df.index, columns=['Evidence'])
evidence_table.loc[:, 'Evidence'] = 'To be populated [@cori2013]'
Markdown(evidence_table.to_markdown())

## References